# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from g_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../WeatherPy/weatherdata.csv")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,Plácido de Castro,100,BR,1603592243,71,-10.34,-67.19,79.65,3.74
1,1,Beringovskiy,99,RU,1603592245,95,63.05,179.32,31.44,13.78
2,2,Haines Junction,100,CA,1603592246,76,60.75,-137.51,25.47,0.94
3,3,Busselton,62,AU,1603591951,48,-33.65,115.33,73.00,4.00
4,4,Cabo San Lucas,5,MX,1603592006,58,22.89,-109.91,84.20,9.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = cities.loc[(cities["Max Temperature"] > 70) & (cities["Max Temperature"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,14,Salinópolis,0,BR,1603592259,84,-0.61,-47.36,79.00,10.29
1,29,Doha,0,QA,1603592276,69,25.29,51.53,75.20,2.24
2,39,São Borja,0,BR,1603592288,84,-28.66,-56.00,70.50,7.58
3,61,Mokobeng,0,BW,1603592312,41,-23.00,27.67,70.59,6.91
4,72,Saint-Philippe,0,RE,1603592325,49,-21.36,55.77,71.60,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
0,14,Salinópolis,0,BR,1603592259,84,-0.61,-47.36,79.00,10.29,Salinópolis
1,29,Doha,0,QA,1603592276,69,25.29,51.53,75.20,2.24,Doha
2,39,São Borja,0,BR,1603592288,84,-28.66,-56.00,70.50,7.58,"São Borja, Rio Grande do Sul"
3,61,Mokobeng,0,BW,1603592312,41,-23.00,27.67,70.59,6.91,Mokobeng
4,72,Saint-Philippe,0,RE,1603592325,49,-21.36,55.77,71.60,5.82,Saint-Philippe


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))